GPT: Generative Pre-Training
- GPT的模型核心组件选择了Transformer，区别于之前的其他一些模型包括ELMo的LSTM。这确实给模型带来了极大的性能和速度提升。
- 为了方便将语言模型的能力transfer到下游的各种任务上去，GPT对模型的输入进行了规范，称为 traversal-style input transformations。
- GPT对词典使用了 bytepair encoding (BPE) subword units来作为基本单元，即不能将句子长度被增加太多而降低模型性能，也能有效减少词典的大小以减少模型参数量


- GPT模型使用经典的two stage training。第一个阶段，将一个大容量的语言模型在很大的无监督语料上进行预训练。第二个阶段，在特定任务的监督数据上进行finetune。
- GPT使用了标准的语言模型目标，优化某个词在其前面k个词出现情况下的条件概率。
- GPT在特定任务的训练时，会把语言模型目标的误差一起加在总误差中联合训练，以提升模型的泛化能力，缓解灾难性遗忘


- `GPT` 使用 `transformer` 的解码模块；`BERT` 则使用其编码模块。
- `GPT` 与传统的语言模型相同，每次输出一个 `token`；然后该 `token` 被加入到输入序列中，新序列作为下一步的输入：`auto-regression`
![](images/gpt-2-autoregression-2.gif)

- `GPT` 使用的 `Masked Self-Attention`，只允许一个位置 `attend` 当前位置及其左边的所有 `token`；`BERT` 则允许左右两边的。


   
1. 给模型输入起始标记 `<s>`，模型输出一个全词汇表的概率分布，概率最高的则为，第一个输出单词；
    - 总是选择概率最高的单词，模型可能会陷入循环，总是输出相同的单词；`top-k` 参数，在概率最高的前 `k` 个单词中采样；
2. 将第一步的输出添加到输入向量中，进行下一步预测

In [ ]:
- 首先在一个嵌入矩阵中查找到 `<s>` 对应的向量；再加上位置向量

**参考链接**
1. http://jalammar.github.io/illustrated-gpt2/
2. https://amaarora.github.io/2020/02/18/annotatedGPT2.html

# GPT 实现

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import numpy as np
import os
import re
import string
import random

## 自注意力

In [3]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        